In [ ]:
!pip install -r requirements.txt

In [ ]:
import json
import  os
import pickle
#from langchain_anthropic import ChatAnthropic
#from langchain_openai import ChatOpenAI
#from langchain_core.output_parsers import StrOutputParser
#from langchain_core.prompts import ChatPromptTemplate
import nest_asyncio
import requests
from transformers import CLIPModel, CLIPProcessor, BlipProcessor, BlipForConditionalGeneration, pipeline
import torch
import asyncio
from openai import AsyncOpenAI
from PIL import Image
import re
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import numpy as np
from requests.auth import HTTPBasicAuth
#import tempfile
import gradio as gr
from pydub import AudioSegment
import markdown

In [ ]:
with open('records.json', 'r', encoding='utf-8-sig') as f:
    records_2 = json.load(f)

In [ ]:
token = "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" # got from groq app
endpoint = "https://api.groq.com/openai/v1"
model_name = "llama-3.3-70b-versatile"

NEO4J_HTTP_URL = "https://e764-2406-7400-43-4409-f0df-66fc-51bf-d50c.ngrok-free.app"
QUERY_ENDPOINT = f"{NEO4J_HTTP_URL}/db/neo4j/tx/commit"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "fashion@123"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda" if torch.cuda.is_available() else "cpu")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
blip_model = BlipForConditionalGeneration.from_pretrained(
        "Salesforce/blip-image-captioning-base",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
# Enable async compatibility for Jupyter
nest_asyncio.apply()

# Azure OpenAI async client setup
client = AsyncOpenAI(
    base_url=endpoint,
    api_key=token,
)

In [ ]:
store_assistant_agent_prompt = """
You are a highly creative and intelligent assistant with a deep understanding of user shopping intent across queries, image descriptions, and voice input.

Your goal is to break down the user’s request into natural language sub-queries, where each sub-query represents a request for one person, and map each sub-query to the most relevant store index group(s).

You will receive:
    • A user query (mandatory)

You may also optionally receive:
    • A list of image descriptions
    • A speech transcription (converted to text)

Instructions:
1. Each sub-query must:
    • Be phrased in natural language, describing a clear clothing or accessory request for one person.
    • Include style, color, or occasion if mentioned.
    • Represent one person only. Do not split intent for the same person into multiple sub-queries.

2. For each sub-query, identify the most likely store index group(s) based on the description.
    • Use one or more of the following exact group names:
        - "Menswear"
        - "Ladieswear"
        - "Baby/Children"
    • You may assign multiple index groups if ambiguous (e.g., “teen” could match both "Menswear" and "Baby/Children").
    • If it’s clearly for a child, use "Baby/Children".
    • Do NOT use "Divided" — it's an internal label.

3. Use optional inputs (image descriptions, speech) only if they enhance understanding.
    • If irrelevant, ignore them and ignore their gender given main gender we get from query if not present then from image or audio.
    ignore gender of voice and images pls ffs gender we get through text or if not given then only speech and images.
    • Never contradict the user's explicit text request based on images or voice.

4. Return a JSON dictionary with sub-queries as keys and a list of index group names as values.

📦 Product Categories (for mapping purposes)

These are the available store index groups:
    1. Menswear – Clothing and accessories intended for men.
    2. Ladieswear – Clothing and accessories intended for women.
    3. Baby/Children – Products for babies, toddlers, and young children.

You must only use these index group names as values in the output.
Please return in json

---

Examples:

Example 1

User Query: “Some Halloween outfits for my husband, cute yellow dress for my daughter, black sunglasses for me”
Output:
{
  "sub_queries": {
    "Find and create a dashing Halloween outfit for an adult male.": ["Menswear"],
    "Find a cute yellow dress for a baby girl.": ["Baby/Children"],
    "Find stylish black sunglasses for a woman.": ["Ladieswear"]
  }
}

Example 2

User Query: “Need party clothes for my wife and some winterwear for my son”
Image Descriptions: ["Image of red blazer and boots for woman", "Image of puffer jacket for boy"]
Output:
{
  "sub_queries": {
    "Find elegant party clothing for a woman.": ["Ladieswear"],
    "Find warm winterwear suitable for a young boy.": ["Baby/Children", "Menswear"]
  }
}

Example 3

User Query: “Looking for college outfit for myself and something cool for my younger brother”
(no image or speech)
Output:
{
  "sub_queries": {
    "Find a casual and trendy outfit suitable for a college-going woman.": ["Ladieswear"],
    "Find a stylish everyday outfit for a teenage boy.": ["Menswear", "Baby/Children"]
  }
}
"""

store_assistant_agent_schema = {
    "type": "function",
    "function": {
        "name": "QueryParserData",
        "description": "Parses user input into structured sub-queries and maps each to a list of relevant index group names for the H&M store.",
        "parameters": {
            "type": "object",
            "properties": {
                "sub_queries": {
                    "type": "object",
                    "description": "A dictionary where each key is a structured sub-query and its value is a list of index group names relevant to that query.",
                    "additionalProperties": {
                        "type": "array",
                        "items": {
                            "type": "string",
                            "description": "An index group name such as 'Men', 'Women', 'Divided', etc."
                        },
                        "description": "List of index group names relevant to the given sub-query."
                    }
                }
            },
            "required": ["sub_queries"],
            "additionalProperties": False
        }
    },
    "strict": True
}

stock_analysis_agent_prompt = f"""
You are an intelligent and highly creative assistant tasked with analyzing a sub-query and mapping it to available product data. You will receive:
	•	A single sub-query string.
	•	A structured JSON variable with indexGroup, product_type_name, and color_group_names.
Always return something most suitable pls be creative and get as many options as you can(product_types and colours).
Your task is to match the sub-query with relevant data from the records and return a structured object with the following:

Return:
	1.	index_group_name (mandatory): One of Menswear, Ladieswear, or Baby/Children.
	2.	product_type_name (array): Exact or closest matches.
	3.	color_group_name (array): Exact match or related colors (generate 3 trendy colors if not specified).

Processing Rules:
	1.	Index Group:
	•	Match exactly to one of Menswear, Ladieswear, Baby/Children.
	•	If no clear match, return an empty object.
	2.	Product Type Name:
	•	Always return as an array.
	•	Prioritize exact match within the same index_group_name; if none, return closest alternatives within the same index_group_name.
	•	If no valid match, return an empty array.
	3.	Color Group Name:
	•	Always return as an array.
	•	If color is mentioned in the sub-query, match exactly or find related colors.
	•	If no color is mentioned, generate 3 trendy colors for the person.
	•	Only return colors relevant to the matched product_type_name under that index_group_name.
	4.	Category Hierarchy:
	•	Only consider products within the matched index_group_name.
	5.	Query Completion:
	•	Correct and complete the query based on available records.

Example input output:
sub_query = "Find yellow dress for a baby girl in Baby/Children"

Output:
      "index_group_name": "Baby/Children",
      "product_type_name": ["Dress", "Tunic", "Blouse"],
      "colour_group_name": ["Yellow", "Light Orange", "Light Pink"]

Input: “Find stylish sunglasses for a woman”
output:
  "index_group_name": "Ladieswear",
  "product_type_name": ["Sunglasses"],
  "colour_group_name": ["Black", "Pink", "Beige"]

Input: “Look for party outfit for my wife”
output:
  "index_group_name": "Ladieswear",
  "product_type_name": ["Dress", "Skirt"],
  "colour_group_name": ["Dark Grey", "Dark Pink", "Beige"]

Input: “Get a Halloween outfit for my teenage daughter”
output:
  "index_group_name": "Baby/Children",
  "product_type_name": ["Dress", "Skirt"],
  "colour_group_name": ["Pink", "Black", "Light Pink"]

Input: “Find a cute skirt for my daughter”
output:
  "index_group_name": "Baby/Children",
  "product_type_name": ["Skirt"],
  "colour_group_name": ["Black", "Light Blue", "Light Pink"]

If something not available don't return anything
colour given should be that of under it's product_type_name
"""

stock_analysis_agent_schema = {
    "type": "function",
    "function": {
        "name": "ExtractedQueryData",
        "description": "Analyzes a single sub-query and returns structured product details based on records.",
        "parameters": {
            "type": "object",
            "properties": {
                "queries": {
                    "type": "array",
                    "description": "List of extracted query details from the sub-query.",
                    "items": {
                        "type": "object",
                        "properties": {
                            "index_group_name": {
                                "type": "string",
                                "description": "Main product category such as Menswear, Ladieswear, etc. This is mandatory."
                            },
                            "product_type_name": {
                                "type": "array",
                                "description": "List of product types derived from the query. Closest match is first.",
                                "items": {
                                    "type": "string"
                                }
                            },
                            "colour_group_name": {
                                "type": "array",
                                "description": "List of colors derived or inferred from the query or records. Always returned as an array.",
                                "items": {
                                    "type": "string"
                                }
                            }
                        }
                    }
                }
            },
            "additionalProperties": False
        }
    },
    "strict":True
}
recommender_agent_prompt = """
You are a highly stylish, intelligent, and persuasive fashion assistant who curates personalized recommendations to help users look and feel their best.

You are provided with:
- The original user query (what they're looking for)
- A list of sub-queries that break down their intent
- Visually matched products (via image understanding)
- Attribute-matched products (via text-based filtering)
- A trend description outlining the latest fashion trends relevant to the user’s query

Your goals are:
1. Begin with a friendly and stylish compliment to the user’s taste based on what they’re looking for.
2. Carefully analyze all options and filter out anything irrelevant, repetitive, or low-quality.
3. Prioritize showing the most fashionable, relevant, and high-quality options that align with the user's stated need and the latest fashion trends from the trend description.
4. If there are multiple relevant items for a sub-query, include the best one first (favoring trend-aligned items), and group additional stylish suggestions near the end.
5. Your tone should feel like a smart, honest fashion advisor — someone who wants the user to feel confident and look great, not just list products.

💡 Don’t just return what was asked. If you spot items that complement the user's taste, enhance their outfit, or align with current trends (e.g., shoes that go with a shirt, accessories for a baby outfit, or trend-inspired additions), feel free to suggest them too — gently and authentically.

For every item you recommend, include:
- Product Name
- Article ID
- Product Type
- Colour Group
- Description
- Source: "image_match" or "text_match"
- Why it's recommended (1–2 sentences, friendly, tailored to the user’s taste, and highlighting alignment with current trends where applicable)

Format your response as clean and easily readable

🧠 Be especially mindful of styling cues in the user query — such as “Levi’s”, “Halloween”, “for my baby”, “formal look”, “something cool”, etc. — and the trend description to justify and personalize your recommendations. Use both to ensure recommendations are stylish and up-to-date.

If multiple items are nearly identical, pick the most suitable one for the main list (prioritizing trend alignment) and include the rest as “additional stylish suggestions” at the end.

You’re not just matching — you’re styling with a focus on the latest trends. Sell the look. 🎯

If you're provided with vague queries like: outfit for Halloween, outfit for a funeral, etc., then make combinations of items given, if possible make multiple; if not, then it's okay with one or just say we don't have them now. Ensure combinations reflect the trend description where applicable.

For complex user queries, respond accordingly, weaving in trend insights to enhance relevance.

No imaginary items; deal only with the ones provided and no ** while responding.
"""

USING TREND ANALYSER-------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# hidden (but to run)
# Define the path to the Pickle file
#pickle_filename = "/content/image_embeddings.pkl"
pickle_filename = "/content/item_data.pkl"

# Load the Pickle file back into item_data
with open(pickle_filename, "rb") as f:
    item_data2 = pickle.load(f)

In [ ]:
async def analyze_fashion_trends(user_query):
    """
    Uses the compound_beta model from Groq to generate a concise description of the latest fashion trends
    related to the user's query, considering synonyms and related terms, in no more than 2 sentences.
    """
    prompt = f"Provide a concise description of the latest fashion trends related to '{user_query}', considering synonyms and related terms, in no more than 2 sentences. Do not include any explanations, reasoning, or intermediate steps."
    response = await client.chat.completions.create(
        model="compound-beta",
        messages=[{"role": "user", "content": prompt}]
    )
    trend_description = response.choices[0].message.content.strip()
    return trend_description

def run_captioning_pipeline(image_paths):
    """
    Generates fashion-focused captions for a list of image paths and returns them as a single formatted string.
    """

    blip_model.eval()

    def clean_caption(caption):
        unwanted_phrases = [
            r"against [a-zA-Z\s]+ background",
            r"in front of [a-zA-Z\s]+",
            r"standing [a-zA-Z\s]+",
            r"sitting [a-zA-Z\s]+",
            r"on [a-zA-Z\s]+ wall",
            r"with [a-zA-Z\s]+ in the background",
            r"the background [a-zA-Z\s]+",
            r"with [a-zA-Z\s]+ hair",
            r"[a-zA-Z\s]+ hair",
            r"with a tattoo [a-zA-Z\s]+",
            r"tattoo on [a-zA-Z\s]+",
            r"[a-zA-Z\s]+ tattoo"
        ]
        for phrase in unwanted_phrases:
            caption = re.sub(phrase, "", caption, flags=re.IGNORECASE)
        return caption.strip().capitalize()

    # Generate captions
    lines = []
    for i, img_path in enumerate(image_paths, start=1):
        try:
            image = Image.open(img_path).convert("RGB")
            inputs = blip_processor(images=image, return_tensors="pt").to(device)

            with torch.no_grad():
                outputs = blip_model.generate(
                    **inputs,
                    max_length=100,
                    num_beams=5,
                    early_stopping=True
                )
            caption = blip_processor.decode(outputs[0], skip_special_tokens=True)
            cleaned = clean_caption(caption)
            lines.append(f"Image {i} Description: {cleaned}")
        except Exception as e:
            lines.append(f"image {i}: [Error] {str(e)}")

    return "\n".join(lines)

def transcribe_audio(audio_path, model_name="openai/whisper-small", language="en"):
    """
    Transcribes speech from an audio file using a specified Whisper model.

    Args:
        audio_path (str): Path to the audio file (e.g., .mp3, .wav).
        model_name (str): The name of the pretrained model to use.
        language (str): Language code for transcription.

    Returns:
        str: Transcribed text.
    """
    asr_pipeline = pipeline("automatic-speech-recognition", model=model_name)
    result = asr_pipeline(audio_path, generate_kwargs={"language": language})
    return result["text"]

async def run_store_assistant_agent(user_query: str, image_indices: list[str] = None):
    if image_indices:
        query_context = (
            f"{user_query}\n\nAvailable image indices: {', '.join(image_indices)}"
        )
    else:
        query_context = user_query

    response = await client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": store_assistant_agent_prompt},
            {"role": "user", "content": query_context}
        ]
    )

    return extract_json(response.choices[0].message.content)

def extract_json(response_text: str):
    """
    Extracts and parses the first valid JSON object found within a string.
    Handles code blocks (```json ... ```), and ignores any text before or after.
    """
    # First: Try to extract JSON inside ```json ... ``` or ``` ... ```
    code_block_match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", response_text, re.DOTALL)
    if code_block_match:
        json_str = code_block_match.group(1)
    else:
        # Fallback: Try to find any JSON-like object in raw text
        brace_match = re.search(r"\{.*\}", response_text, re.DOTALL)
        if brace_match:
            json_str = brace_match.group(0)
        else:
            raise ValueError("No JSON object found in response.")

    # Attempt to parse it
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"JSON parsing failed: {e}")

async def run_stock_analysis_agent(sub_query_kv: dict):
    """
    sub_query_kv: A dict like { "Find something for a man": ["Menswear", "Baby/Children"] }
    """

    # Get indexGroup filters from the value
    index_groups = list(sub_query_kv.values())[0]

    # Filter only the relevant indexGroup sections
    relevant_product_info = [
        section for section in records_2 if section["indexGroup"] in index_groups
    ]

    # Compose the system prompt
    filtered_prompt = stock_analysis_agent_prompt + "\n\n---\n\n" + \
        f"📦 Relevant Product Records below (based on index groups in this request):\n\n" + \
        json.dumps(relevant_product_info, indent=2)

    #print(filtered_prompt)
    # Send the filtered context to GPT
    response = await client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": filtered_prompt},
            {"role": "user", "content": json.dumps(sub_query_kv)}
        ],
        tools=[stock_analysis_agent_schema],
        tool_choice="required",
    )

    tool_call = response.choices[0].message.tool_calls[0]
    parsed = json.loads(tool_call.function.arguments)
    return parsed

def build_subquery_cypher(sub_query):
    """
    Example sub_query:
    {
      "index_group_name": "Menswear",
      "product_type_name": ["Pants", "Chinos", "Trousers"],
      "colour_group_name": ["Green", "Olive", "Forest Green"]
    }
    """

    index_group_name = sub_query.get("index_group_name", "")
    product_types = sub_query.get("product_type_name", [])
    colour_groups = sub_query.get("colour_group_name", [])

    # Build parameterized Cypher
    statement = """
    MATCH (i:Item)
    WHERE i.index_group_name = $indexGroupName
      AND (
           size($productTypes) = 0
           OR i.product_type_name IN $productTypes
      )
      AND (
           size($colourGroups) = 0
           OR i.colour_group_name IN $colourGroups
      )
    RETURN i.article_id         AS article_id,
           i.prod_name          AS prod_name,
           i.colour_group_name  AS colour_group_name,
           i.product_type_name  AS product_type_name,
           i.index_group_name   AS index_group_name
    """

    return {
        "statement": statement,
        "parameters": {
            "indexGroupName": index_group_name,
            "productTypes": product_types,
            "colourGroups": colour_groups
        }
    }

def fetch_item_embeddings():
    query = {
        "statements": [
            {
                "statement": """
                MATCH (i:Item)
                RETURN i.article_id AS article_id,
                       i.prod_name AS prod_name,
                       i.colour_group_name AS colour_group_name,
                       i.department_name AS department_name,
                       i.detail_desc AS detail_desc,
                       i.garment_group_name AS garment_group_name,
                       i.index_group_name AS index_group_name,
                       i.perceived_colour_master_name AS perceived_colour_master_name,
                       i.product_group_name AS product_group_name,
                       i.image_embedding AS image_embedding
                """
            }
        ]
    }

    response = requests.post(
        QUERY_ENDPOINT,
        json=query,
        auth=HTTPBasicAuth(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

    if response.status_code == 200:
        try:
            data = response.json()["results"][0]["data"]
            return [
                {
                    "article_id": row["row"][0],
                    "prod_name": row["row"][1],
                    "colour_group_name": row["row"][2],
                    "department_name": row["row"][3],
                    "detail_desc": row["row"][4],
                    "garment_group_name": row["row"][5],
                    "index_group_name": row["row"][6],
                    "perceived_colour_master_name": row["row"][7],
                    "product_group_name": row["row"][8],
                    "image_embedding": np.array(row["row"][9]) if row["row"][9] else None
                }
                for row in data
            ]
        except (json.JSONDecodeError, KeyError, IndexError):
            print("\n🚨 JSON Decode Error! Invalid Neo4j Response")
            print(response.text[:1000])
            return []
    else:
        print(f"🚨 Error fetching embeddings: {response.status_code}, {response.text}")
        return []


def get_top_k_similar_items_from_images(image_paths, k=5):
    # Step 1: Compute CLIP embeddings
    image_embeddings = []
    for path in image_paths:
        image = Image.open(path).convert("RGB")
        inputs = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = clip_model.get_image_features(**inputs)
            image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
        image_embeddings.append(image_features.cpu().numpy()[0])

    query_embedding = np.mean(image_embeddings, axis=0)

    # Step 2: Fetch all item vectors
    item_data = item_data2
    item_vectors = [item["image_embedding"] for item in item_data if item["image_embedding"] is not None]
    item_metadata = [item for item in item_data if item["image_embedding"] is not None]

    if not item_vectors:
        print("❌ No valid image embeddings found in Neo4j.")
        return []

    # Step 3: Cosine similarity
    sims = cosine_similarity([query_embedding], item_vectors)[0]
    top_indices = np.argsort(sims)[::-1][:k]

    top_results = []
    for i in top_indices:
        meta = item_metadata[i]
        top_results.append({
            "article_id": meta["article_id"],
            "prod_name": meta["prod_name"],
            "colour_group_name": meta["colour_group_name"],
            "department_name": meta["department_name"],
            "detail_desc": meta["detail_desc"],
            "garment_group_name": meta["garment_group_name"],
            "index_group_name": meta["index_group_name"],
            "perceived_colour_master_name": meta["perceived_colour_master_name"],
            "product_group_name": meta["product_group_name"],
            "similarity_score": float(sims[i])
        })

    return top_results


def build_subquery_cypher(sub_query, exclude_colours=False):
    """
    Builds a Cypher query for fetching products based on a sub-query.
    If `exclude_colours` is True, excludes the colour_groups condition.
    """
    index_group_name = sub_query.get("index_group_name", "")
    product_types = sub_query.get("product_type_name", [])
    colour_groups = sub_query.get("colour_group_name", [])

    # If we're excluding colour_groups, we won't add that part of the query
    colour_groups_condition = ""
    parameters = {
        "indexGroupName": index_group_name,
        "productTypes": product_types
    }

    if not exclude_colours:
        colour_groups_condition = """
        AND (
            size($colourGroups) = 0
            OR i.colour_group_name IN $colourGroups
        )
        """
        parameters["colourGroups"] = colour_groups

    statement = f"""
    MATCH (i:Item)
    WHERE i.index_group_name = $indexGroupName
      AND (
           size($productTypes) = 0
           OR i.product_type_name IN $productTypes
      )
      {colour_groups_condition}
    RETURN i.article_id         AS article_id,
           i.prod_name          AS prod_name,
           i.colour_group_name  AS colour_group_name,
           i.product_type_name  AS product_type_name,
           i.index_group_name   AS index_group_name,
           i.detail_desc        AS detail_desc,
           i.department_name    AS department_name,
           i.garment_group_name AS garment_group_name,
           i.perceived_colour_master_name AS perceived_colour_master_name,
           i.product_group_name AS product_group_name
    LIMIT 50
    """

    return {
        "statement": statement,
        "parameters": parameters
    }

def text_recommender_agent(final_result):
    """
    Runs Cypher query for each valid sub-query from stock_analysis_agent output.
    A sub-query is only valid if it has a non-empty index_group_name and product_type_name.
    """
    results = []

    for sub_query_text, query_obj in final_result.items():
        query_items = query_obj.get("queries", [])
        for sub_query in query_items:
            index_group = sub_query.get("index_group_name", "")
            product_types = sub_query.get("product_type_name", [])

            # ✅ Skip if missing required fields
            if not index_group or not product_types:
                print(f"⏩ Skipping sub-query '{sub_query_text}' due to missing index_group_name or product_type_name.")
                continue

            # Build and send Cypher with colour_groups
            query_payload = {
                "statements": [build_subquery_cypher(sub_query)]
            }

            response = requests.post(
                QUERY_ENDPOINT,
                json=query_payload,
                auth=HTTPBasicAuth(NEO4J_USERNAME, NEO4J_PASSWORD)
            )

            # If no results, try without colour_groups
            if response.status_code == 200:
                try:
                    data = response.json()["results"][0]["data"]
                    if not data:  # If no data returned, try again without colour_groups
                        print(f"❌ No data found for '{sub_query_text}', retrying without colour_groups...")
                        query_payload["statements"] = [build_subquery_cypher(sub_query, exclude_colours=True)]
                        response = requests.post(
                            QUERY_ENDPOINT,
                            json=query_payload,
                            auth=HTTPBasicAuth(NEO4J_USERNAME, NEO4J_PASSWORD)
                        )

                        if response.status_code == 200:
                            data = response.json()["results"][0]["data"]
                        else:
                            print(f"❌ Retry failed for '{sub_query_text}': {response.status_code} - {response.text}")
                            continue  # Skip this sub-query if still no result

                    for row in data:
                        results.append({
                            "sub_query": sub_query_text,
                            "article_id": row["row"][0],
                            "prod_name": row["row"][1],
                            "colour_group_name": row["row"][2],
                            "product_type_name": row["row"][3],
                            "index_group_name": row["row"][4],
                            "detail_desc": row["row"][5],
                            "department_name": row["row"][6],
                            "garment_group_name": row["row"][7],
                            "perceived_colour_master_name": row["row"][8],
                            "product_group_name": row["row"][9]
                        })
                except Exception as e:
                    print(f"⚠️ Error parsing result for sub-query '{sub_query_text}': {e}")
            else:
                print(f"❌ Neo4j query failed for '{sub_query_text}': {response.status_code} - {response.text}")

    return results

async def run_recommender_agent(
    user_query: str,
    sub_queries: list,
    top_matches: list,
    text_matches: list
):
    """
    Generates curated fashion recommendations considering the latest trends.
    """
    trend_description = await analyze_fashion_trends(user_query)
    messages = [
        {"role": "system", "content": recommender_agent_prompt},
        {"role": "user", "content": json.dumps({
            "user_query": user_query,
            "sub_queries": sub_queries,
            "image_matches": top_matches,
            "text_matches": text_matches,
            "trend_description": trend_description
        }, indent=2)}
    ]
    response = await client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    print("\n💡 Final Curated Recommendations:\n" + "-"*60)
    return response.choices[0].message.content

USING THE EMBEDDING FROM DATA.PKL-------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# hidden (but to run)
# Define the path to the Pickle file
#pickle_filename = "/content/image_embeddings.pkl"
pickle_filename = "/content/item_data.pkl"

# Load the Pickle file back into item_data
with open(pickle_filename, "rb") as f:
    item_data2 = pickle.load(f)

In [ ]:
def run_captioning_pipeline(image_paths):
    """
    Generates fashion-focused captions for a list of image paths and returns them as a single formatted string.
    """

    blip_model.eval()

    def clean_caption(caption):
        unwanted_phrases = [
            r"against [a-zA-Z\s]+ background",
            r"in front of [a-zA-Z\s]+",
            r"standing [a-zA-Z\s]+",
            r"sitting [a-zA-Z\s]+",
            r"on [a-zA-Z\s]+ wall",
            r"with [a-zA-Z\s]+ in the background",
            r"the background [a-zA-Z\s]+",
            r"with [a-zA-Z\s]+ hair",
            r"[a-zA-Z\s]+ hair",
            r"with a tattoo [a-zA-Z\s]+",
            r"tattoo on [a-zA-Z\s]+",
            r"[a-zA-Z\s]+ tattoo"
        ]
        for phrase in unwanted_phrases:
            caption = re.sub(phrase, "", caption, flags=re.IGNORECASE)
        return caption.strip().capitalize()

    # Generate captions
    lines = []
    for i, img_path in enumerate(image_paths, start=1):
        try:
            image = Image.open(img_path).convert("RGB")
            inputs = blip_processor(images=image, return_tensors="pt").to(device)

            with torch.no_grad():
                outputs = blip_model.generate(
                    **inputs,
                    max_length=100,
                    num_beams=5,
                    early_stopping=True
                )
            caption = blip_processor.decode(outputs[0], skip_special_tokens=True)
            cleaned = clean_caption(caption)
            lines.append(f"Image {i} Description: {cleaned}")
        except Exception as e:
            lines.append(f"image {i}: [Error] {str(e)}")

    return "\n".join(lines)

def transcribe_audio(audio_path, model_name="openai/whisper-small", language="en"):
    """
    Transcribes speech from an audio file using a specified Whisper model.

    Args:
        audio_path (str): Path to the audio file (e.g., .mp3, .wav).
        model_name (str): The name of the pretrained model to use.
        language (str): Language code for transcription.

    Returns:
        str: Transcribed text.
    """
    asr_pipeline = pipeline("automatic-speech-recognition", model=model_name)
    result = asr_pipeline(audio_path, generate_kwargs={"language": language})
    return result["text"]

async def run_store_assistant_agent(user_query: str, image_indices: list[str] = None):
    if image_indices:
        query_context = (
            f"{user_query}\n\nAvailable image indices: {', '.join(image_indices)}"
        )
    else:
        query_context = user_query

    response = await client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": store_assistant_agent_prompt},
            {"role": "user", "content": query_context}
        ]
    )

    return extract_json(response.choices[0].message.content)

def extract_json(response_text: str):
    """
    Extracts and parses the first valid JSON object found within a string.
    Handles code blocks (```json ... ```), and ignores any text before or after.
    """
    # First: Try to extract JSON inside ```json ... ``` or ``` ... ```
    code_block_match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", response_text, re.DOTALL)
    if code_block_match:
        json_str = code_block_match.group(1)
    else:
        # Fallback: Try to find any JSON-like object in raw text
        brace_match = re.search(r"\{.*\}", response_text, re.DOTALL)
        if brace_match:
            json_str = brace_match.group(0)
        else:
            raise ValueError("No JSON object found in response.")

    # Attempt to parse it
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"JSON parsing failed: {e}")

async def run_stock_analysis_agent(sub_query_kv: dict):
    """
    sub_query_kv: A dict like { "Find something for a man": ["Menswear", "Baby/Children"] }
    """

    # Get indexGroup filters from the value
    index_groups = list(sub_query_kv.values())[0]

    # Filter only the relevant indexGroup sections
    relevant_product_info = [
        section for section in records_2 if section["indexGroup"] in index_groups
    ]

    # Compose the system prompt
    filtered_prompt = stock_analysis_agent_prompt + "\n\n---\n\n" + \
        f"📦 Relevant Product Records below (based on index groups in this request):\n\n" + \
        json.dumps(relevant_product_info, indent=2)

    #print(filtered_prompt)
    # Send the filtered context to GPT
    response = await client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": filtered_prompt},
            {"role": "user", "content": json.dumps(sub_query_kv)}
        ],
        tools=[stock_analysis_agent_schema],
        tool_choice="required",
    )

    tool_call = response.choices[0].message.tool_calls[0]
    parsed = json.loads(tool_call.function.arguments)
    return parsed

def build_subquery_cypher(sub_query):
    """
    Example sub_query:
    {
      "index_group_name": "Menswear",
      "product_type_name": ["Pants", "Chinos", "Trousers"],
      "colour_group_name": ["Green", "Olive", "Forest Green"]
    }
    """

    index_group_name = sub_query.get("index_group_name", "")
    product_types = sub_query.get("product_type_name", [])
    colour_groups = sub_query.get("colour_group_name", [])

    # Build parameterized Cypher
    statement = """
    MATCH (i:Item)
    WHERE i.index_group_name = $indexGroupName
      AND (
           size($productTypes) = 0
           OR i.product_type_name IN $productTypes
      )
      AND (
           size($colourGroups) = 0
           OR i.colour_group_name IN $colourGroups
      )
    RETURN i.article_id         AS article_id,
           i.prod_name          AS prod_name,
           i.colour_group_name  AS colour_group_name,
           i.product_type_name  AS product_type_name,
           i.index_group_name   AS index_group_name
    """

    return {
        "statement": statement,
        "parameters": {
            "indexGroupName": index_group_name,
            "productTypes": product_types,
            "colourGroups": colour_groups
        }
    }

def fetch_item_embeddings():
    query = {
        "statements": [
            {
                "statement": """
                MATCH (i:Item)
                RETURN i.article_id AS article_id,
                       i.prod_name AS prod_name,
                       i.colour_group_name AS colour_group_name,
                       i.department_name AS department_name,
                       i.detail_desc AS detail_desc,
                       i.garment_group_name AS garment_group_name,
                       i.index_group_name AS index_group_name,
                       i.perceived_colour_master_name AS perceived_colour_master_name,
                       i.product_group_name AS product_group_name,
                       i.image_embedding AS image_embedding
                """
            }
        ]
    }

    response = requests.post(
        QUERY_ENDPOINT,
        json=query,
        auth=HTTPBasicAuth(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

    if response.status_code == 200:
        try:
            data = response.json()["results"][0]["data"]
            return [
                {
                    "article_id": row["row"][0],
                    "prod_name": row["row"][1],
                    "colour_group_name": row["row"][2],
                    "department_name": row["row"][3],
                    "detail_desc": row["row"][4],
                    "garment_group_name": row["row"][5],
                    "index_group_name": row["row"][6],
                    "perceived_colour_master_name": row["row"][7],
                    "product_group_name": row["row"][8],
                    "image_embedding": np.array(row["row"][9]) if row["row"][9] else None
                }
                for row in data
            ]
        except (json.JSONDecodeError, KeyError, IndexError):
            print("\n🚨 JSON Decode Error! Invalid Neo4j Response")
            print(response.text[:1000])
            return []
    else:
        print(f"🚨 Error fetching embeddings: {response.status_code}, {response.text}")
        return []


def get_top_k_similar_items_from_images(image_paths, k=5):
    # Step 1: Compute CLIP embeddings
    image_embeddings = []
    for path in image_paths:
        image = Image.open(path).convert("RGB")
        inputs = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = clip_model.get_image_features(**inputs)
            image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
        image_embeddings.append(image_features.cpu().numpy()[0])

    query_embedding = np.mean(image_embeddings, axis=0)

    # Step 2: Fetch all item vectors
    item_data = item_data2
    item_vectors = [item["image_embedding"] for item in item_data if item["image_embedding"] is not None]
    item_metadata = [item for item in item_data if item["image_embedding"] is not None]

    if not item_vectors:
        print("❌ No valid image embeddings found in Neo4j.")
        return []

    # Step 3: Cosine similarity
    sims = cosine_similarity([query_embedding], item_vectors)[0]
    top_indices = np.argsort(sims)[::-1][:k]

    top_results = []
    for i in top_indices:
        meta = item_metadata[i]
        top_results.append({
            "article_id": meta["article_id"],
            "prod_name": meta["prod_name"],
            "colour_group_name": meta["colour_group_name"],
            "department_name": meta["department_name"],
            "detail_desc": meta["detail_desc"],
            "garment_group_name": meta["garment_group_name"],
            "index_group_name": meta["index_group_name"],
            "perceived_colour_master_name": meta["perceived_colour_master_name"],
            "product_group_name": meta["product_group_name"],
            "similarity_score": float(sims[i])
        })

    return top_results


def build_subquery_cypher(sub_query, exclude_colours=False):
    """
    Builds a Cypher query for fetching products based on a sub-query.
    If `exclude_colours` is True, excludes the colour_groups condition.
    """
    index_group_name = sub_query.get("index_group_name", "")
    product_types = sub_query.get("product_type_name", [])
    colour_groups = sub_query.get("colour_group_name", [])

    # If we're excluding colour_groups, we won't add that part of the query
    colour_groups_condition = ""
    parameters = {
        "indexGroupName": index_group_name,
        "productTypes": product_types
    }

    if not exclude_colours:
        colour_groups_condition = """
        AND (
            size($colourGroups) = 0
            OR i.colour_group_name IN $colourGroups
        )
        """
        parameters["colourGroups"] = colour_groups

    statement = f"""
    MATCH (i:Item)
    WHERE i.index_group_name = $indexGroupName
      AND (
           size($productTypes) = 0
           OR i.product_type_name IN $productTypes
      )
      {colour_groups_condition}
    RETURN i.article_id         AS article_id,
           i.prod_name          AS prod_name,
           i.colour_group_name  AS colour_group_name,
           i.product_type_name  AS product_type_name,
           i.index_group_name   AS index_group_name,
           i.detail_desc        AS detail_desc,
           i.department_name    AS department_name,
           i.garment_group_name AS garment_group_name,
           i.perceived_colour_master_name AS perceived_colour_master_name,
           i.product_group_name AS product_group_name
    LIMIT 50
    """

    return {
        "statement": statement,
        "parameters": parameters
    }

def text_recommender_agent(final_result):
    """
    Runs Cypher query for each valid sub-query from stock_analysis_agent output.
    A sub-query is only valid if it has a non-empty index_group_name and product_type_name.
    """
    results = []

    for sub_query_text, query_obj in final_result.items():
        query_items = query_obj.get("queries", [])
        for sub_query in query_items:
            index_group = sub_query.get("index_group_name", "")
            product_types = sub_query.get("product_type_name", [])

            # ✅ Skip if missing required fields
            if not index_group or not product_types:
                print(f"⏩ Skipping sub-query '{sub_query_text}' due to missing index_group_name or product_type_name.")
                continue

            # Build and send Cypher with colour_groups
            query_payload = {
                "statements": [build_subquery_cypher(sub_query)]
            }

            response = requests.post(
                QUERY_ENDPOINT,
                json=query_payload,
                auth=HTTPBasicAuth(NEO4J_USERNAME, NEO4J_PASSWORD)
            )

            # If no results, try without colour_groups
            if response.status_code == 200:
                try:
                    data = response.json()["results"][0]["data"]
                    if not data:  # If no data returned, try again without colour_groups
                        print(f"❌ No data found for '{sub_query_text}', retrying without colour_groups...")
                        query_payload["statements"] = [build_subquery_cypher(sub_query, exclude_colours=True)]
                        response = requests.post(
                            QUERY_ENDPOINT,
                            json=query_payload,
                            auth=HTTPBasicAuth(NEO4J_USERNAME, NEO4J_PASSWORD)
                        )

                        if response.status_code == 200:
                            data = response.json()["results"][0]["data"]
                        else:
                            print(f"❌ Retry failed for '{sub_query_text}': {response.status_code} - {response.text}")
                            continue  # Skip this sub-query if still no result

                    for row in data:
                        results.append({
                            "sub_query": sub_query_text,
                            "article_id": row["row"][0],
                            "prod_name": row["row"][1],
                            "colour_group_name": row["row"][2],
                            "product_type_name": row["row"][3],
                            "index_group_name": row["row"][4],
                            "detail_desc": row["row"][5],
                            "department_name": row["row"][6],
                            "garment_group_name": row["row"][7],
                            "perceived_colour_master_name": row["row"][8],
                            "product_group_name": row["row"][9]
                        })
                except Exception as e:
                    print(f"⚠️ Error parsing result for sub-query '{sub_query_text}': {e}")
            else:
                print(f"❌ Neo4j query failed for '{sub_query_text}': {response.status_code} - {response.text}")

    return results

async def run_recommender_agent(
    user_query: str,
    sub_queries: list,
    top_matches: list,
    text_matches: list
):

    messages = [
        {"role": "system", "content": recommender_agent_prompt},
        {"role": "user", "content": json.dumps({
            "user_query": user_query,
            "sub_queries": sub_queries,
            "image_matches": top_matches,
            "text_matches": text_matches
        }, indent=2)}
    ]

    response = await client.chat.completions.create(
        model=model_name,
        messages=messages
    )

    print("\n💡 Final Curated Recommendations:\n" + "-"*60)

    return response.choices[0].message.content

USING EMBEDDINGS DIRECTLY FROM NEO4J------------------------------------------------------------------------------------------

In [ ]:
def run_captioning_pipeline(image_paths):
    """
    Generates fashion-focused captions for a list of image paths and returns them as a single formatted string.
    """

    blip_model.eval()

    def clean_caption(caption):
        unwanted_phrases = [
            r"against [a-zA-Z\s]+ background",
            r"in front of [a-zA-Z\s]+",
            r"standing [a-zA-Z\s]+",
            r"sitting [a-zA-Z\s]+",
            r"on [a-zA-Z\s]+ wall",
            r"with [a-zA-Z\s]+ in the background",
            r"the background [a-zA-Z\s]+",
            r"with [a-zA-Z\s]+ hair",
            r"[a-zA-Z\s]+ hair",
            r"with a tattoo [a-zA-Z\s]+",
            r"tattoo on [a-zA-Z\s]+",
            r"[a-zA-Z\s]+ tattoo"
        ]
        for phrase in unwanted_phrases:
            caption = re.sub(phrase, "", caption, flags=re.IGNORECASE)
        return caption.strip().capitalize()

    # Generate captions
    lines = []
    for i, img_path in enumerate(image_paths, start=1):
        try:
            image = Image.open(img_path).convert("RGB")
            inputs = blip_processor(images=image, return_tensors="pt").to(device)

            with torch.no_grad():
                outputs = blip_model.generate(
                    **inputs,
                    max_length=100,
                    num_beams=5,
                    early_stopping=True
                )
            caption = blip_processor.decode(outputs[0], skip_special_tokens=True)
            cleaned = clean_caption(caption)
            lines.append(f"Image {i} Description: {cleaned}")
        except Exception as e:
            lines.append(f"image {i}: [Error] {str(e)}")

    return "\n".join(lines)

def transcribe_audio(audio_path, model_name="openai/whisper-small", language="en"):
    """
    Transcribes speech from an audio file using a specified Whisper model.

    Args:
        audio_path (str): Path to the audio file (e.g., .mp3, .wav).
        model_name (str): The name of the pretrained model to use.
        language (str): Language code for transcription.

    Returns:
        str: Transcribed text.
    """
    asr_pipeline = pipeline("automatic-speech-recognition", model=model_name)
    result = asr_pipeline(audio_path, generate_kwargs={"language": language})
    return result["text"]

async def run_store_assistant_agent(user_query: str, image_indices: list[str] = None):
    if image_indices:
        query_context = (
            f"{user_query}\n\nAvailable image indices: {', '.join(image_indices)}"
        )
    else:
        query_context = user_query

    response = await client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": store_assistant_agent_prompt},
            {"role": "user", "content": query_context}
        ]
    )

    return extract_json(response.choices[0].message.content)

def extract_json(response_text: str):
    """
    Extracts and parses the first valid JSON object found within a string.
    Handles code blocks (```json ... ```), and ignores any text before or after.
    """
    # First: Try to extract JSON inside ```json ... ``` or ``` ... ```
    code_block_match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", response_text, re.DOTALL)
    if code_block_match:
        json_str = code_block_match.group(1)
    else:
        # Fallback: Try to find any JSON-like object in raw text
        brace_match = re.search(r"\{.*\}", response_text, re.DOTALL)
        if brace_match:
            json_str = brace_match.group(0)
        else:
            raise ValueError("No JSON object found in response.")

    # Attempt to parse it
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"JSON parsing failed: {e}")

async def run_stock_analysis_agent(sub_query_kv: dict):
    """
    sub_query_kv: A dict like { "Find something for a man": ["Menswear", "Baby/Children"] }
    """

    # Get indexGroup filters from the value
    index_groups = list(sub_query_kv.values())[0]

    # Filter only the relevant indexGroup sections
    relevant_product_info = [
        section for section in records_2 if section["indexGroup"] in index_groups
    ]

    # Compose the system prompt
    filtered_prompt = stock_analysis_agent_prompt + "\n\n---\n\n" + \
        f"📦 Relevant Product Records below (based on index groups in this request):\n\n" + \
        json.dumps(relevant_product_info, indent=2)

    #print(filtered_prompt)
    # Send the filtered context to GPT
    response = await client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": filtered_prompt},
            {"role": "user", "content": json.dumps(sub_query_kv)}
        ],
        tools=[stock_analysis_agent_schema],
        tool_choice="required",
    )

    tool_call = response.choices[0].message.tool_calls[0]
    parsed = json.loads(tool_call.function.arguments)
    return parsed

def build_subquery_cypher(sub_query):
    """
    Example sub_query:
    {
      "index_group_name": "Menswear",
      "product_type_name": ["Pants", "Chinos", "Trousers"],
      "colour_group_name": ["Green", "Olive", "Forest Green"]
    }
    """

    index_group_name = sub_query.get("index_group_name", "")
    product_types = sub_query.get("product_type_name", [])
    colour_groups = sub_query.get("colour_group_name", [])

    # Build parameterized Cypher
    statement = """
    MATCH (i:Item)
    WHERE i.index_group_name = $indexGroupName
      AND (
           size($productTypes) = 0
           OR i.product_type_name IN $productTypes
      )
      AND (
           size($colourGroups) = 0
           OR i.colour_group_name IN $colourGroups
      )
    RETURN i.article_id         AS article_id,
           i.prod_name          AS prod_name,
           i.colour_group_name  AS colour_group_name,
           i.product_type_name  AS product_type_name,
           i.index_group_name   AS index_group_name
    """

    return {
        "statement": statement,
        "parameters": {
            "indexGroupName": index_group_name,
            "productTypes": product_types,
            "colourGroups": colour_groups
        }
    }

def fetch_item_embeddings():
    query = {
        "statements": [
            {
                "statement": """
                MATCH (i:Item)
                RETURN i.article_id AS article_id,
                       i.prod_name AS prod_name,
                       i.colour_group_name AS colour_group_name,
                       i.department_name AS department_name,
                       i.detail_desc AS detail_desc,
                       i.garment_group_name AS garment_group_name,
                       i.index_group_name AS index_group_name,
                       i.perceived_colour_master_name AS perceived_colour_master_name,
                       i.product_group_name AS product_group_name,
                       i.image_embedding AS image_embedding
                """
            }
        ]
    }

    response = requests.post(
        QUERY_ENDPOINT,
        json=query,
        auth=HTTPBasicAuth(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

    if response.status_code == 200:
        try:
            data = response.json()["results"][0]["data"]
            return [
                {
                    "article_id": row["row"][0],
                    "prod_name": row["row"][1],
                    "colour_group_name": row["row"][2],
                    "department_name": row["row"][3],
                    "detail_desc": row["row"][4],
                    "garment_group_name": row["row"][5],
                    "index_group_name": row["row"][6],
                    "perceived_colour_master_name": row["row"][7],
                    "product_group_name": row["row"][8],
                    "image_embedding": np.array(row["row"][9]) if row["row"][9] else None
                }
                for row in data
            ]
        except (json.JSONDecodeError, KeyError, IndexError):
            print("\n🚨 JSON Decode Error! Invalid Neo4j Response")
            print(response.text[:1000])
            return []
    else:
        print(f"🚨 Error fetching embeddings: {response.status_code}, {response.text}")
        return []

#CHANGE: ----------------------
# Fetch data once at the start (add this at the top of your notebook)
item_data2 = fetch_item_embeddings()
#------------------------------

# Your existing get_top_k_similar_items_from_images function
def get_top_k_similar_items_from_images(image_paths, k=5):
    # Step 1: Compute CLIP embeddings
    image_embeddings = []
    for path in image_paths:
        image = Image.open(path).convert("RGB")
        inputs = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = clip_model.get_image_features(**inputs)
            image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
        image_embeddings.append(image_features.cpu().numpy()[0])

    query_embedding = np.mean(image_embeddings, axis=0)

    # Step 2: Fetch all item vectors
    item_data = item_data2
    item_vectors = [item["image_embedding"] for item in item_data if item["image_embedding"] is not None]
    item_metadata = [item for item in item_data if item["image_embedding"] is not None]

    if not item_vectors:
        print("❌ No valid image embeddings found in Neo4j.")
        return []

    # Step 3: Cosine similarity
    sims = cosine_similarity([query_embedding], item_vectors)[0]
    top_indices = np.argsort(sims)[::-1][:k]

    top_results = []
    for i in top_indices:
        meta = item_metadata[i]
        top_results.append({
            "article_id": meta["article_id"],
            "prod_name": meta["prod_name"],
            "colour_group_name": meta["colour_group_name"],
            "department_name": meta["department_name"],
            "detail_desc": meta["detail_desc"],
            "garment_group_name": meta["garment_group_name"],
            "index_group_name": meta["index_group_name"],
            "perceived_colour_master_name": meta["perceived_colour_master_name"],
            "product_group_name": meta["product_group_name"],
            "similarity_score": float(sims[i])
        })

    return top_results


def build_subquery_cypher(sub_query, exclude_colours=False):
    """
    Builds a Cypher query for fetching products based on a sub-query.
    If `exclude_colours` is True, excludes the colour_groups condition.
    """
    index_group_name = sub_query.get("index_group_name", "")
    product_types = sub_query.get("product_type_name", [])
    colour_groups = sub_query.get("colour_group_name", [])

    # If we're excluding colour_groups, we won't add that part of the query
    colour_groups_condition = ""
    parameters = {
        "indexGroupName": index_group_name,
        "productTypes": product_types
    }

    if not exclude_colours:
        colour_groups_condition = """
        AND (
            size($colourGroups) = 0
            OR i.colour_group_name IN $colourGroups
        )
        """
        parameters["colourGroups"] = colour_groups

    statement = f"""
    MATCH (i:Item)
    WHERE i.index_group_name = $indexGroupName
      AND (
           size($productTypes) = 0
           OR i.product_type_name IN $productTypes
      )
      {colour_groups_condition}
    RETURN i.article_id         AS article_id,
           i.prod_name          AS prod_name,
           i.colour_group_name  AS colour_group_name,
           i.product_type_name  AS product_type_name,
           i.index_group_name   AS index_group_name,
           i.detail_desc        AS detail_desc,
           i.department_name    AS department_name,
           i.garment_group_name AS garment_group_name,
           i.perceived_colour_master_name AS perceived_colour_master_name,
           i.product_group_name AS product_group_name
    LIMIT 50
    """

    return {
        "statement": statement,
        "parameters": parameters
    }

def text_recommender_agent(final_result):
    """
    Runs Cypher query for each valid sub-query from stock_analysis_agent output.
    A sub-query is only valid if it has a non-empty index_group_name and product_type_name.
    """
    results = []

    for sub_query_text, query_obj in final_result.items():
        query_items = query_obj.get("queries", [])
        for sub_query in query_items:
            index_group = sub_query.get("index_group_name", "")
            product_types = sub_query.get("product_type_name", [])

            # ✅ Skip if missing required fields
            if not index_group or not product_types:
                print(f"⏩ Skipping sub-query '{sub_query_text}' due to missing index_group_name or product_type_name.")
                continue

            # Build and send Cypher with colour_groups
            query_payload = {
                "statements": [build_subquery_cypher(sub_query)]
            }

            response = requests.post(
                QUERY_ENDPOINT,
                json=query_payload,
                auth=HTTPBasicAuth(NEO4J_USERNAME, NEO4J_PASSWORD)
            )

            # If no results, try without colour_groups
            if response.status_code == 200:
                try:
                    data = response.json()["results"][0]["data"]
                    if not data:  # If no data returned, try again without colour_groups
                        print(f"❌ No data found for '{sub_query_text}', retrying without colour_groups...")
                        query_payload["statements"] = [build_subquery_cypher(sub_query, exclude_colours=True)]
                        response = requests.post(
                            QUERY_ENDPOINT,
                            json=query_payload,
                            auth=HTTPBasicAuth(NEO4J_USERNAME, NEO4J_PASSWORD)
                        )

                        if response.status_code == 200:
                            data = response.json()["results"][0]["data"]
                        else:
                            print(f"❌ Retry failed for '{sub_query_text}': {response.status_code} - {response.text}")
                            continue  # Skip this sub-query if still no result

                    for row in data:
                        results.append({
                            "sub_query": sub_query_text,
                            "article_id": row["row"][0],
                            "prod_name": row["row"][1],
                            "colour_group_name": row["row"][2],
                            "product_type_name": row["row"][3],
                            "index_group_name": row["row"][4],
                            "detail_desc": row["row"][5],
                            "department_name": row["row"][6],
                            "garment_group_name": row["row"][7],
                            "perceived_colour_master_name": row["row"][8],
                            "product_group_name": row["row"][9]
                        })
                except Exception as e:
                    print(f"⚠️ Error parsing result for sub-query '{sub_query_text}': {e}")
            else:
                print(f"❌ Neo4j query failed for '{sub_query_text}': {response.status_code} - {response.text}")

    return results

async def run_recommender_agent(
    user_query: str,
    sub_queries: list,
    top_matches: list,
    text_matches: list
):

    messages = [
        {"role": "system", "content": recommender_agent_prompt},
        {"role": "user", "content": json.dumps({
            "user_query": user_query,
            "sub_queries": sub_queries,
            "image_matches": top_matches,
            "text_matches": text_matches
        }, indent=2)}
    ]

    response = await client.chat.completions.create(
        model=model_name,
        messages=messages
    )

    print("\n💡 Final Curated Recommendations:\n" + "-"*60)

    return response.choices[0].message.content

MAIN CODE------------------------------------------------------------------

In [ ]:
async def main():
    user_query = "i want these for me im a man"
    image_indices = ['hmm.jpeg']
    audio_path = "trail1.mp3"
    top_matches = []  # make sure this is always defined

    # Step 0: Append voice transcription if available
    if audio_path:
        try:
            transcription = transcribe_audio(audio_path)
            user_query += f"\nAdditional speech input from user: {transcription}"
        except Exception as e:
            print(f"[Voice Error] Could not transcribe audio: {e}")

    # Step 1: Append image captions if available
    if image_indices:
        try:
            image_descriptions = run_captioning_pipeline(image_indices)
            user_query += f"\nThe user also uploaded images. Here's what they look like:\n{image_descriptions}"
        except Exception as e:
            print(f"[Image Error] Could not caption images: {e}")

    # Step 2: Get sub-queries from store assistant
    store_result = await run_store_assistant_agent(user_query, image_indices)
    sub_queries = store_result.get("sub_queries", {})

    print("=== Sub-queries ===")
    for sub_query, index_groups in sub_queries.items():
        print(f"- {sub_query} -> {index_groups}")

    # Step 3: Loop through sub-queries and call stock_analysis_agent
    final_result = {}
    for sub_query, index_groups in sub_queries.items():
        single_pair = {sub_query: index_groups}
        stock_response = await run_stock_analysis_agent(single_pair)
        final_result[sub_query] = stock_response

    print("\n=== Final Stock Analysis ===")
    print(json.dumps(final_result, indent=2))

     # Step 4: Display latest fashion trends
    print("\n👗 Latest Fashion Trends for Your Query:\n" + "-"*60)
    try:
        trend_description = await analyze_fashion_trends(user_query)
        print(f"Trends: {trend_description}")
    except Exception as e:
        print(f"[Trend Error] Could not fetch fashion trends: {e}")

    # Step 5: Image similarity matches (optional)
    if image_indices:
        print("\n📸 Top 5 Visually Similar Products (based on uploaded images):\n" + "-"*60)
        top_matches = get_top_k_similar_items_from_images(image_indices, k=5)
        for i, item in enumerate(top_matches, 1):
            print(f"\n🔹 Items {i}")
            print(f"   • Product Name      : {item['prod_name']}")
            print(f"   • Description       : {item['detail_desc']}")
            print(f"   • Article ID        : {item['article_id']}")
            print(f"   • Colour Group      : {item['colour_group_name']}")
            print(f"   • Perceived Colour  : {item['perceived_colour_master_name']}")
            print(f"   • Product Group     : {item['product_group_name']}")
            print(f"   • Garment Group     : {item['garment_group_name']}")
            print(f"   • Department        : {item['department_name']}")
            print(f"   • Index Group       : {item['index_group_name']}")
            print(f"   • 🔁 Similarity Score : {item['similarity_score']:.4f}")

    # Step 6: Text-based matching
    print("\n🧠 Text-Based Recommendations from Neo4j (per sub-query):\n" + "-"*60)
    text_matches = text_recommender_agent(final_result)
    if not text_matches:
        print("No matches found for the sub-queries.")
    else:
        for i, item in enumerate(text_matches, 1):
            print(f"\n🔸 Match {i} for sub-query: '{item['sub_query']}'")
            print(f"   • Product Name      : {item['prod_name']}")
            print(f"   • Description       : {item['detail_desc']}")
            print(f"   • Article ID        : {item['article_id']}")
            print(f"   • Colour Group      : {item['colour_group_name']}")
            print(f"   • Perceived Colour  : {item['perceived_colour_master_name']}")
            print(f"   • Product Type      : {item['product_type_name']}")
            print(f"   • Product Group     : {item['product_group_name']}")
            print(f"   • Garment Group     : {item['garment_group_name']}")
            print(f"   • Department        : {item['department_name']}")
            print(f"   • Index Group       : {item['index_group_name']}")

    # Step 7: Final curated recommendations
    final_recommendations = await run_recommender_agent(
        user_query=user_query,
        sub_queries=sub_queries,
        top_matches=top_matches,
        text_matches=text_matches
    )
    print(final_recommendations)

# ✅ Now run it:
await main()

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


=== Sub-queries ===
- Find a suitable outfit for a 21-year-old male to wear to a graduation ceremony. -> ['Menswear']

=== Final Stock Analysis ===
{
  "Find a suitable outfit for a 21-year-old male to wear to a graduation ceremony.": {
    "queries": [
      {
        "index_group_name": "Menswear",
        "product_type_name": [
          "Blazer",
          "Shirt",
          "Trousers"
        ],
        "colour_group_name": [
          "Black",
          "Dark Grey",
          "Dark Blue"
        ]
      }
    ]
  }
}

👗 Latest Fashion Trends for Your Query:
------------------------------------------------------------
Trends: As a 21-year-old male, the latest fashion trends for graduation include stylish and modern outfits such as a tailored navy blue or charcoal grey suit with a crisp white dress shirt and a slim tie. A grey shirt paired with black pants, as seen in the uploaded image, is also a smart-casual alternative that embodies current fashion trends for men, suitable for a

In [ ]:
# --------------------------------------------------
# Our main wrapper in Gradio
# --------------------------------------------------
async def main_wrapper(user_query, image_inputs, audio_file):
    """
    user_query: Text from the user
    image_inputs: List of file objects from `gr.File(file_types=['image'], multiple=True)`
    audio_file:   Path to an audio file from `gr.Audio(type='filepath')`
    """

    # 1) Convert Gradio's image inputs to local file paths
    if image_inputs is None:
        image_paths = []
    else:
        # image_inputs is a list of dicts with 'name', 'data', etc.
        # We only need the 'name' key for the local file path
        image_paths = [img.name for img in image_inputs] if image_inputs else []

    # 2) Handle Audio (transcription)
    if audio_file:
        try:
            # Use from_file instead of from_wav
            audio_segment = AudioSegment.from_file(audio_file)

            # Convert to MP3
            mp3_path = audio_file + ".mp3"
            audio_segment.export(mp3_path, format="mp3")

            # Transcribe
            transcription = transcribe_audio(mp3_path)
            user_query += f"\nAdditional speech input from user: {transcription}"

            # Cleanup if desired
            if os.path.exists(mp3_path):
                os.remove(mp3_path)

        except Exception as e:
            user_query += f"\n[Voice Error]: {e}"

    # 3) Caption images
    if image_paths:
        try:
            image_descriptions = run_captioning_pipeline(image_paths)
            user_query += f"\nThe user uploaded images:\n{image_descriptions}"
        except Exception as e:
            user_query += f"\n[Image Captioning Error]: {e}"

    # 4) Run store assistant
    store_result = await run_store_assistant_agent(user_query, image_paths)
    sub_queries = store_result.get("sub_queries", {})

    # 5) Stock analysis
    final_result = {}
    for sub_query, index_groups in sub_queries.items():
        single_pair = {sub_query: index_groups}
        stock_response = await run_stock_analysis_agent(single_pair)
        final_result[sub_query] = stock_response

    # 6) Image similarity
    top_matches = []
    if image_paths:
        top_matches = get_top_k_similar_items_from_images(image_paths, k=5)

    # 7) Text recommendations
    text_matches = text_recommender_agent(final_result)

    # 8) Final recommendations
    final_recommendations = await run_recommender_agent(
        user_query=user_query,
        sub_queries=sub_queries,
        top_matches=top_matches,
        text_matches=text_matches
    )

    # 9) Build an HTML block from your results
    markdown_content = f"""# 👟 Fashion Assistant Recommendations

**Final Recommendations**: {final_recommendations}

---
_Made with ❤️ by your style assistant AI._
"""
    markdown_html = markdown.markdown(markdown_content)
    styled_html = f"""
<div style="
    padding: 1rem;
    border-radius: 12px;
    background-color: #4f46e5;
    max-height: 500px;
    overflow-y: auto;
    font-family: 'Segoe UI', sans-serif;
    line-height: 1.6;
">
  <style>
    div#fashion-output * {{
        color: #ffffff !important;
    }}
  </style>
  <div id="fashion-output">
    {markdown_html}
  </div>
</div>
"""
    return styled_html

# --------------------------------------------------
# Gradio Interface
# --------------------------------------------------
with gr.Blocks(theme=gr.themes.Soft()) as fashion_ui:
    gr.Markdown("""
    <div style='text-align: center; margin-bottom: 1rem;'>
      <h1 style='color:#ffffff; font-size: 2rem;'>Your Personalised Fashion Assistant Agent</h1>
      <p style='font-size: 1.1rem; color: #111;'>Drop images, speak your style, or type it — get curated fashion picks instantly.</p>
    </div>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("""
            ### 🧾 Tell Us Your Style
            Drop everything here — type, upload, or record. We'll figure it out.
            """)

            with gr.Row(equal_height=True):
                user_query = gr.Textbox(
                    label="Text Prompt",
                    placeholder="e.g. A party outfit for my wife and a hoodie for my son",
                    lines=2
                )

            with gr.Row(equal_height=True):
                # Multiple images
                image_inputs = gr.File(
                    label="Upload Images",
                    file_types=["image"],
                    file_count="multiple"
                )
                # Audio as filepath
                audio_input = gr.Audio(
                    label="Record Voice",
                    type="filepath"
                )

            submit_btn = gr.Button("🔍 Get Recommendations", size="lg")

        with gr.Column(scale=1):
            gr.Markdown("### 💡 Curated Fashion Suggestions")
            output_box = gr.HTML()
            gr.Markdown("")

    # Link the submit button to our async function
    submit_btn.click(
        fn=main_wrapper,
        inputs=[user_query, image_inputs, audio_input],
        outputs=output_box
    )

# Note: If your environment doesn't like async in Gradio,
# you can test in a standard Python script or remove `async` usage.

fashion_ui.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3a89b3417010f9813c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(



💡 Final Curated Recommendations:
------------------------------------------------------------


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(



💡 Final Curated Recommendations:
------------------------------------------------------------


TESTING

In [ ]:
from openai import AsyncOpenAI

import os

token = "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"
endpoint = "https://api.groq.com/openai/v1"
model_name = "llama-3.3-70b-versatile"

client = AsyncOpenAI(
    base_url=endpoint,
    api_key=token,
)

async def main():
    response = await client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is the capital of Spain?"}
        ],
        model=model_name
    )
    print(response.choices[0].message.content)

# Run the async main function
asyncio.run(main())


The capital of Spain is Madrid.


TESTING COMPOUND MODEL

In [ ]:
client  = AsyncOpenAI(base_url = "https://api.groq.com/openai/v1", api_key="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" )


# Assuming 'client' is an asynchronous client (e.g., groq.AsyncGroq) properly initialized with a valid API key
async def fetch_bitcoin_value():
    try:
        # Await the asynchronous API call to get the streaming response
        completion = await client.chat.completions.create(
            model="compound-beta",  # Replace with a valid model if "compound-beta" is incorrect (e.g., "mixtral-8x7b-32768" for Groq)
            messages=[
                {
                    'role': 'user',
                    'content': 'What is the current value of 0.38474 Bitcoin based on CoinGecko?'
                }
            ],
            temperature=1,
            max_completion_tokens=1024,
            top_p=1,
            stream=True,  # Enable streaming for real-time response
            stop=None,
        )
        # Iterate over the streaming response using async for
        async for chunk in completion:
            print(chunk.choices[0].delta.content or "", end="")
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the async function in a Jupyter notebook cell
await fetch_bitcoin_value()

To find the current value of 0.38474 Bitcoin based on CoinGecko, I first searched for the current price of Bitcoin in USD on CoinGecko. The search results provided various prices for Bitcoin, but I needed to find a reliable and current price.

The most relevant result seemed to be: 
URL: https://www.coingecko.com/en/coins/bitcoin/usd
Content: Convert Bitcoin (BTC) to US Dollar (USD). Coins:16,967. Exchanges:1,270. Market Cap: $3.133T1.8%.24h Vol: $79 

The conversion rate of Bitcoin (BTC) to USD is $94.304,39 for every 1 BTC.

Using this conversion rate, I calculated the current value of 0.38474 Bitcoin as follows:

```python
current_bitcoin_price = 94304.39
amount_bitcoin = 0.38474
current_value = current_bitcoin_price * amount_bitcoin
print(current_value)
```

The output of this calculation is:
36282.6710086

Therefore, the current value of 0.38474 Bitcoin is approximately $36,282.67 USD based on CoinGecko.